In [1]:
from pyspark.sql import SparkSession

# Khởi tạo Spark Session
spark = SparkSession.builder \
    .appName("HotelDataProcessing") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9000") \
    .config("spark.hadoop.ipc.maximum.data.length", "1073741824") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

In [2]:
cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")

You are working with 1 core(s)


In [3]:
namenode_uri = "hdfs://localhost:9000"
facility_path = f"{namenode_uri}/data_lake/raw/hotels/facility"
faqs_path = f"{namenode_uri}/data_lake/raw/hotels/faqs"
images_path = f"{namenode_uri}/data_lake/raw/hotels/images"
policy_path = f"{namenode_uri}/data_lake/raw/hotels/policy"
reviews_path = f"{namenode_uri}/data_lake/raw/hotels/reviews"
rooms_path = f"{namenode_uri}/data_lake/raw/hotels/room"

In [4]:
df_facility = spark.read.csv(f"{facility_path}/*.csv", header=True, inferSchema=True)
df_faqs = spark.read.csv(f"{faqs_path}/*.csv", header=True, inferSchema=True)
df_images = spark.read.csv(f"{images_path}/*.csv", header=True, inferSchema=True)
df_policy = spark.read.csv(f"{policy_path}/*.csv", header=True, inferSchema=True)
df_reviews = spark.read.csv(f"{reviews_path}/*.csv", header=True, inferSchema=True)
df_rooms = spark.read.csv(f"{rooms_path}/*.csv", header=True, inferSchema=True)

In [5]:
def print_schema(df):
    return df.printSchema()
    
print(df_facility.count())
print_schema(df_facility)

print(df_faqs.count())
print_schema(df_faqs)

print(df_images.count())
print_schema(df_images)

print(df_policy.count())
print_schema(df_policy)

print(df_reviews.count())
print_schema(df_reviews)

print(df_rooms.count())
print_schema(df_rooms)

16356
root
 |-- Url: string (nullable = true)
 |-- Facilities: string (nullable = true)

16356
root
 |-- Url: string (nullable = true)
 |-- FAQs: string (nullable = true)

2378
root
 |-- url: string (nullable = true)
 |-- images: string (nullable = true)
 |-- images_num: integer (nullable = true)

16356
root
 |-- url: string (nullable = true)
 |-- policy_info: string (nullable = true)

110073
root
 |-- username: string (nullable = true)
 |-- score: string (nullable = true)
 |-- comment: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- stay_at: string (nullable = true)
 |-- hotel_url: string (nullable = true)
 |-- review_count: string (nullable = true)

4582
root
 |-- _c0: integer (nullable = true)
 |-- url: string (nullable = true)
 |-- price_discount: string (nullable = true)
 |-- original_price: string (nullable = true)
 |-- discount: double (nullable = true)



In [6]:
df_rooms.show(5)

+---+--------------------+--------------+--------------+--------+
|_c0|                 url|price_discount|original_price|discount|
+---+--------------------+--------------+--------------+--------+
|  0|https://www.booki...| VND 1.570.000| VND 1.570.000|     0.0|
|  1|https://www.booki...| VND 2.500.000| VND 2.500.000|     0.0|
|  2|https://www.booki...|   VND 312.300| VND 1.041.000|    70.0|
|  3|https://www.booki...|   VND 349.800| VND 1.166.000|    70.0|
|  4|https://www.booki...|   VND 320.000|   VND 320.000|     0.0|
+---+--------------------+--------------+--------------+--------+
only showing top 5 rows



In [7]:
df_reviews.show(5)

+--------+-----+--------------------+--------------------+------------+--------------------+------------+
|username|score|             comment|          created_at|     stay_at|           hotel_url|review_count|
+--------+-----+--------------------+--------------------+------------+--------------------+------------+
|     Năm|  8,0|{'title': 'Rất tố...|ngày 2 tháng 9 nă...|tháng 9/2024|https://www.booki...|           1|
|     Mai|  7,0|{'title': 'Khách ...|ngày 15 tháng 2 n...|Tháng 2/2022|https://www.booki...|          20|
|    Linh|  7,0|{'title': 'Tốt', ...|ngày 23 tháng 6 n...|tháng 6/2024|https://www.booki...|          20|
|    Thoa|  8,0|{'title': 'được',...|ngày 7 tháng 7 nă...|tháng 7/2022|https://www.booki...|          20|
|      Tư|  6,0|{'title': 'Tạm, đ...|ngày 12 tháng 4 n...|tháng 4/2024|https://www.booki...|          20|
+--------+-----+--------------------+--------------------+------------+--------------------+------------+
only showing top 5 rows



In [8]:
df_reviews.collect()[0]['comment']

"{'title': 'Rất tốt', 'positive': None, 'negative': None}"

In [9]:
df_facility.collect()[0]['Facilities']

"{'best_facilities': ['WiFi miễn phí', 'Chỗ đỗ xe miễn phí', 'Bữa sáng', 'Nhà hàng', 'Dịch vụ phòng', 'Phòng không hút thuốc', 'Lễ tân 24 giờ', 'Phòng gia đình', 'Xe đưa đón sân bay', 'Hồ bơi ngoài trời'], 'all_facilities': {'Phòng tắm': ['Phòng tắm riêng', 'Chậu rửa vệ sinh (bidet)', 'Khăn tắm', 'Dép', 'Máy sấy tóc', 'Vòi sen', 'Nhà vệ sinh', 'Giấy vệ sinh', 'Đồ vệ sinh cá nhân miễn phí'], 'Phòng ngủ': ['Đồng hồ báo thức', 'Phòng thay quần áo', 'Tủ hoặc phòng để quần áo', 'Ra trải giường'], 'Tầm nhìn': ['Nhìn ra thành phố', 'Hướng nhìn sân trong', 'Nhìn ra vườn', 'Tầm nhìn ra khung cảnh'], 'Ngoài trời': ['Tiện nghi BBQ', 'Sân hiên', 'Sân vườn', 'Tiện nghi BBQ - Phụ phí', 'Sân hiên phơi nắng', 'Bàn ghế ngoài trời', 'Sân trong', 'Khu vực ăn uống ngoài trời'], 'Nhà bếp': ['Ấm đun nước điện', 'Sản phẩm lau rửa', 'Bàn ăn', 'Bếp chung', 'Máy giặt'], 'Tiện ích trong phòng': ['Giá treo quần áo', 'Ổ điện gần giường'], 'Hoạt động': ['Xe đạp', 'Tour/lớp về văn hó

In [10]:
df_final = df_reviews.join(df_rooms, df_reviews.hotel_url == df_rooms.url, how='left') \
                     .join(df_images, on='url', how='left')

df_final = df_final.drop('username', 'hotel_url', 'images', 'created_at', '_c0')
print(df_final.count())  

211060


In [11]:
df_final.show(5)

+--------------------+-----+--------------------+------------+------------+--------------+--------------+--------+----------+
|                 url|score|             comment|     stay_at|review_count|price_discount|original_price|discount|images_num|
+--------------------+-----+--------------------+------------+------------+--------------+--------------+--------+----------+
|https://www.booki...|  8,0|{'title': 'Rất tố...|tháng 9/2024|           1|   VND 450.000|   VND 450.000|     0.0|         7|
|https://www.booki...|  7,0|{'title': 'Khách ...|Tháng 2/2022|          20|   VND 900.000|   VND 900.000|     0.0|        36|
|https://www.booki...|  7,0|{'title': 'Khách ...|Tháng 2/2022|          20|   VND 700.000|   VND 700.000|     0.0|        36|
|https://www.booki...|  7,0|{'title': 'Tốt', ...|tháng 6/2024|          20|   VND 900.000|   VND 900.000|     0.0|        36|
|https://www.booki...|  7,0|{'title': 'Tốt', ...|tháng 6/2024|          20|   VND 700.000|   VND 700.000|     0.0|    